In [2]:
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import matplotlib.pyplot as plt
import multiprocessing as mp
%matplotlib inline
from sklearn.metrics import pairwise_distances_argmin_min
import nltk
#nltk.download('punkt')
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:

# coding: utf-8

# In[3]:


pool = mp.Pool(mp.cpu_count())
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import nltk
from nltk.corpus import wordnet as wn
from nltk import word_tokenize

import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


# In[4]:


import nltk
from nltk import word_tokenize
import pandas as pd


def tokenize(q1, q2):
    """
        q1 and q2 are sentences/questions. Function returns a list of tokens for both.
    """
    return word_tokenize(q1), word_tokenize(q2)


def posTag(q1, q2):
    """
        q1 and q2 are lists. Function returns a list of POS tagged tokens for both.
    """
    return nltk.pos_tag(q1), nltk.pos_tag(q2)


def stemmer(tag_q1, tag_q2):
    """
        tag_q = tagged lists. Function returns a stemmed list.
    """

    stem_q1 = []
    stem_q2 = []

    for token in tag_q1:
        stem_q1.append(stem(token))

    for token in tag_q2:
        stem_q2.append(stem(token))

    return stem_q1, stem_q2


# In[5]:


class Lesk(object):

    def __init__(self, sentence):
        self.sentence = sentence
        self.meanings = {}
        for word in sentence:
            self.meanings[word] = ''

    def getSenses(self, word):
        # print word
        return wn.synsets(word.lower())

    def getGloss(self, senses):

        gloss = {}

        for sense in senses:
            gloss[sense.name()] = []

        for sense in senses:
            gloss[sense.name()] += word_tokenize(sense.definition())

        return gloss
    def getAll(self, word):
        senses = self.getSenses(word)

        if senses == []:
            return {word.lower(): senses}

        return self.getGloss(senses)

    def Score(self, set1, set2):
        # Base
        overlap = 0

        # Step
        for word in set1:
            if word in set2:
                overlap += 1

        return overlap

    def overlapScore(self, word1, word2):

        gloss_set1 = self.getAll(word1)
        if self.meanings[word2] == '':
            gloss_set2 = self.getAll(word2)
        else:
            # print 'here'
            gloss_set2 = self.getGloss([wn.synset(self.meanings[word2])])

        # print gloss_set2
        score = {}
        for i in gloss_set1.keys():
            score[i] = 0
            for j in gloss_set2.keys():
                score[i] += self.Score(gloss_set1[i], gloss_set2[j])

        bestSense = None
        max_score = 0
        for i in gloss_set1.keys():
            if score[i] > max_score:
                max_score = score[i]
                bestSense = i

        return bestSense, max_score

    def lesk(self, word, sentence):
        maxOverlap = 0
        context = sentence
        word_sense = []
        meaning = {}

        senses = self.getSenses(word)

        for sense in senses:
            meaning[sense.name()] = 0

        for word_context in context:
            if not word == word_context:
                score = self.overlapScore(word, word_context)
                if score[0] == None:
                    continue
                meaning[score[0]] += score[1]

        if senses == []:
            return word, None, None
        self.meanings[word] = max(meaning.keys(), key=lambda x: meaning[x])

        return word, self.meanings[word], wn.synset(self.meanings[word]).definition()


# In[7]:


import math
import numpy as np
from scipy import spatial
from nltk.corpus import wordnet as wn
from nltk.metrics import edit_distance

def path(set1, set2):
    return wn.path_similarity(set1, set2)


def wup(set1, set2):
    return wn.wup_similarity(set1, set2)


def edit(word1, word2):
    if float(edit_distance(word1, word2)) == 0.0:
        return 0.0
    return 1.0 / float(edit_distance(word1, word2))


# In[10]:


def computePath(q1, q2):

    R = np.zeros((len(q1), len(q2)))

    for i in range(len(q1)):
        for j in range(len(q2)):
            if q1[i][1] == None or q2[j][1] == None:
                sim = edit(q1[i][0], q2[j][0])
            else:
                sim = path(wn.synset(q1[i][1]), wn.synset(q2[j][1]))

            if sim == None:
                sim = edit(q1[i][0], q2[j][0])

            R[i, j] = sim

    # print R

    return R
def computeWup(q1, q2):

    R = np.zeros((len(q1), len(q2)))

    for i in range(len(q1)):
        for j in range(len(q2)):
            if q1[i][1] == None or q2[j][1] == None:
                sim = edit(q1[i][0], q2[j][0])
            else:
                sim = wup(wn.synset(q1[i][1]), wn.synset(q2[j][1]))

            if sim == None:
                sim = edit(q1[i][0], q2[j][0])

            R[i, j] = sim

    # print R

    return R

def overallSim(q1, q2, R):

    sum_X = 0.0
    sum_Y = 0.0

    for i in range(len(q1)):
        max_i = 0.0
        for j in range(len(q2)):
            if R[i, j] > max_i:
                max_i = R[i, j]
        sum_X += max_i

    for i in range(len(q1)):
        max_j = 0.0
        for j in range(len(q2)):
            if R[i, j] > max_j:
                max_j = R[i, j]
        sum_Y += max_j
        
    if (float(len(q1)) + float(len(q2))) == 0.0:
        return 0.0
        
    overall = (sum_X + sum_Y) / (2 * (float(len(q1)) + float(len(q2))))

    return overall


    tokens_q1, tokens_q2 = tokenize(q1, q2)
    # stem_q1, stem_q2 = stemmer(tokens_q1, tokens_q2)
    tag_q1, tag_q2 = posTag(tokens_q1, tokens_q2)

    sentence = []
    for i, word in enumerate(tag_q1):
        if 'NN' in word[1] or 'JJ' in word[1] or 'VB' in word[1]:
            sentence.append(word[0])

    sense1 = Lesk(sentence)
    sentence1Means = []
    for word in sentence:
        sentence1Means.append(sense1.lesk(word, sentence))

    sentence = []
    for i, word in enumerate(tag_q2):
        if 'NN' in word[1] or 'JJ' in word[1] or 'VB' in word[1]:
            sentence.append(word[0])

    sense2 = Lesk(sentence)
    sentence2Means = []
    for word in sentence:
        sentence2Means.append(sense2.lesk(word, sentence))
    # for i, word in enumerate(sentence1Means):
    #     print sentence1Means[i][0], sentence2Means[i][0]

    R1 = computePath(sentence1Means, sentence2Means)
    R2 = computeWup(sentence1Means, sentence2Means)

    R = (R1 + R2) / 2

    # print R

    return overallSim(sentence1Means, sentence2Means, R)
def semanticSimilarity(q1, q2):

    tokens_q1, tokens_q2 = tokenize(q1, q2)
    # stem_q1, stem_q2 = stemmer(tokens_q1, tokens_q2)
    tag_q1, tag_q2 = posTag(tokens_q1, tokens_q2)

    sentence = []
    for i, word in enumerate(tag_q1):
        if 'NN' in word[1] or 'JJ' in word[1] or 'VB' in word[1]:
            sentence.append(word[0])

    sense1 = Lesk(sentence)
    sentence1Means = []
    for word in sentence:
        sentence1Means.append(sense1.lesk(word, sentence))
    sentence = []
    for i, word in enumerate(tag_q2):
        if 'NN' in word[1] or 'JJ' in word[1] or 'VB' in word[1]:
            sentence.append(word[0])

    sense2 = Lesk(sentence)
    sentence2Means = []
    for word in sentence:
        sentence2Means.append(sense2.lesk(word, sentence))
    # for i, word in enumerate(sentence1Means):
    #     print sentence1Means[i][0], sentence2Means[i][0]

    R1 = computePath(sentence1Means, sentence2Means)
    R2 = computeWup(sentence1Means, sentence2Means)

    R = (R1 + R2) / 2

    # print R
    return overallSim(sentence1Means, sentence2Means, R)
import nltk
STOP_WORDS = nltk.corpus.stopwords.words()
def clean_sentence(val):
    "remove chars that are not letters or numbers, downcase, then remove stop words"
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    sentence = sentence.split(" ")

    for word in list(sentence):
        if word in STOP_WORDS:
            sentence.remove(word)

    sentence = " ".join(sentence)
    return sentence

def build_summary(text):
    #find sentences to extract for summary
    sents_for_sum = find_similarities(text.split(". "))
    #sort the sentences
    sort = sorted(sents_for_sum)
    #display which sentences have been selected
    print(sort)
    
    sent_list = sent_tokenize(text)
    #print number of sentences in full article
    print(len(sent_list))
    
    
    #extract the selected sentences from the original text
    sents = []
    for i in sort:
        sents.append(sent_list[i].replace('\n', ''))
    
    #join sentences together for final output
    summary = ' '.join(sents)
    return summary

def find_similarities(text):
    #tokenize sentences
    sentences = text
    #set stop words
    stops = list(set(stopwords.words('english'))) + list(punctuation)
    
    #vectorize sentences and remove stop words
    vectorizer = TfidfVectorizer(stop_words = stops)
    #print(vectorizer)
    #transform using TFIDF vectorizer
    #print(sentences)
    trsfm=vectorizer.fit_transform(sentences)

    #creat df for input article
    text_df = pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names(),index=sentences)
    
    #declare how many sentences to use in summary
    num_sentences = text_df.shape[0]
    num_summary_sentences = int(np.ceil(num_sentences**.5))
    #print(trsfm)
    #find cosine similarity for all sentence pairs
    similarities = cosine_similarity(trsfm, trsfm)
    lesksimilarities = []
    sentenceno = 0
    #sms = [[0*num_sentences]*num_sentences]*num_sentences
    
    for sentence in sentences:
        temp = []
        #sentenceno2 = sentenceno
        #print("First: "+ sentence)
        similarities= dosentences(sentence,sentences,sentenceno,similarities)
        sentenceno +=1
    print(similarities)
    avgs = []
    for i in similarities:
        avgs.append(i.mean())
     
    #find index values of the sentences to be used for summary
    top_idx = np.argsort(avgs)[-num_summary_sentences:]
    #print(avgs)
    return top_idx

def dosentences(sentence,sentences,sentenceno,similarities):

    sentenceno2 = sentenceno
    #print("First: "+ sentence)
    for i in range(0,sentenceno):
        #print(str(sentenceno)+"Sims: " + str(similarities[i][sentenceno]))
        similarities[sentenceno][i] =similarities[i][sentenceno]

    #print("Sims: " + str(similarities[0][sentenceno]))
    #print(similarities[sentenceno])
    for sentence2 in sentences[sentenceno:len(sentences)]:
        #print(sentence2)
        if(sentence2!=sentence):
            sim = semanticSimilarity(sentence2,sentence)
            #sms[sentenceno][sentenceno2]=sim
            similarities[sentenceno][sentenceno2]+=sim
        sentenceno2+=1

    return(similarities)
def dosentences2(sentence,sentences,sentenceno,similarities):

    sentenceno2 = sentenceno
    #print("First: "+ sentence)
    for i in range(0,sentenceno):
        #print(str(sentenceno)+"Sims: " + str(similarities[i][sentenceno]))
        similarities[sentenceno][i] =similarities[i][sentenceno]

    #print("Sims: " + str(similarities[0][sentenceno]))
    #print(similarities[sentenceno])
    similarities = semsimilarity(sentence,sentences,sentenceno,similarities,sentenceno2)
    [pool.apply(semsimilarity, args=(sentence,sentences,sentenceno,similarities)) for sentence2 in sentences[sentenceno:len(sentences)]]
    return(similarities)
def semsimilarity(sentence,sentences,sentenceno,similarities,sentenceno2):
        #print(sentence2)
    if(sentence2!=sentence):
        sim = semanticSimilarity(sentence2,sentence)
        #sms[sentenceno][sentenceno2]=sim
        similarities[sentenceno][sentenceno2]+=sim
    sentenceno2+=1
    return(similarities)

Use sample text to determine the sentences to be extracted for the summary

In [7]:
sample_3 = """Product.  The Conatix cysana (CYberSecurity ANAlytics) insider threat detection system applies artificial intelligence to analyze everything on the screen of every employee endpoint (computer, tablet, phone) on a bank, company or government agency IT network, second by second, to stop insider fraud as early as possible.

Differentiation.  Conatix cysana innovates in 3 key areas: Data, Analytics and Interface. We stream the most granular data available, everything on the screen of every endpoint on a bank IT network even with tens of thousands of employees, second by second, all text, image, video in applications and in the browser – unlike other cybersecurity systems that start with highly summarized network-level data. We solve the problem of continuous massive fast data streaming and real-time analysis, on-premise behind the bank’s own firewall.  For the interface, rather than issuing thousands of unprioritized text alerts per day to system administrators, we visualize the changing health of the IT network in 3D so both departmental and individual levels can be tracked visually.  Use case video: https://vimeo.com/163300481  Pitch video: https://vimeo.com/213777145

Value.  Conatix cysana increases transparency, integrity and security in financial services, business and government.  It reduces the opportunity for moneylaundering, bribery, theft and fraud and increases visibility into the IT network to help organizations function more effectively.  cysana catches suspicious activity by users of an enterprise network that other systems will miss, and detects problems other systems will detect earlier and faster than they do, providing the highest available level of protection of proprietary, confidential and customer data based on unprecedented real-time visibility into activity on the enterprise network – affording detailed insight into the behavior and risk level of users, workgroups, departments and office locations.  

Benefits.  Extremely high protection of client identifying data (CID) and other proprietary data in a bank or company, which fulfills Swiss FINMA conditions and which may limit corporate liability under GDPR.  Quick start availability as a preinstalled and preconfigured virtual appliance.  Simple and limited number of required installation steps and internet connections: No server-side installation required.  Easy rollout department by department or even device by device as required.  Plug and play: No alteration of the company server or network is required to install the client-side interceptor.  Real-time decryption of encrypted websites immediately and reliably within the browser on each individual device.  Reliable interceptor which no action by individual users can circumvent.  Automatic monitoring of each individual machine and user on a granular basis, so it is never necessary to separate out data streams from multiple machines.  Immediate foolproof traceability of captured data to the specific computer it was intercepted from.  The Conatix client-side interceptor does everything that a server-side interceptor does (fast setup, real-time warning, flexible integration with multiple data streaming and analysis systems, availability as a virtual appliance) and more, making it a value-adding enhancement to server-side or network-side interception.  If suspicious activities occur, alerts and access denial are immediate.  The system is autonomous but also integratable with existing SOC, SIEM and cyber measures.
 
Market.  “Insider threat via a company’s own employees (and contractors and vendors) is one of the largest unsolved issues in cybersecurity.” McKinsey & Company, 2018.  “The role that insiders play in the vulnerability of all sizes of corporations is massive and growing.” Harvard Business Review, 2016.  The global cybersecurity market is expected to exceed $243 billion by 2023. MarketWatch, 2019.  
 """

This function will call upon the find_similarities() function and will then arrange the sentences in the proper order.

<h1>Examples

In [4]:
sentence = "It's been a long time no see."
sentence2= "It's been a long time no see"
semanticSimilarity(sentence2,sentence)

0.5

In [4]:
a= find_similarities("""It's been a long time no see. What is going on my dude. It's been a long day, without you my friend. I can't wait to see you again.""".split(". "))

1Sims: 0.1111111111111111
2Sims: 0.35141718303052394
2Sims: 0.16333943833943834
3Sims: 0.32591355218485263
3Sims: 0.0
3Sims: 0.0
[[1.         0.11111111 0.35141718 0.32591355]
 [0.11111111 1.         0.16333944 0.        ]
 [0.35141718 0.16333944 1.         0.        ]
 [0.32591355 0.         0.         1.        ]]


In [6]:
a

array([3, 0])

In [12]:
text= """It's been a long time no see. What is going on my dude. It's been a long day, without you my friend. I can't wait to see you again."""
build_summary(text+text+text)

1Sims: 0.1111111111111111
2Sims: 0.31596034702801157
2Sims: 0.16333943833943834
3Sims: 1.3558631551260676
3Sims: 0.07147435897435897
3Sims: 0.17917309872877996
4Sims: 0.1111111111111111
4Sims: 1.0000000000000002
4Sims: 0.12380952380952381
4Sims: 0.1111111111111111
5Sims: 0.31596034702801157
5Sims: 0.16333943833943834
5Sims: 1.0
5Sims: 0.24980663186231308
5Sims: 0.16333943833943834
6Sims: 1.3558631551260676
6Sims: 0.07147435897435897
6Sims: 0.17917309872877996
6Sims: 1.0
6Sims: 0.07147435897435897
6Sims: 0.17917309872877996
7Sims: 0.1111111111111111
7Sims: 1.0000000000000002
7Sims: 0.12380952380952381
7Sims: 0.1111111111111111
7Sims: 1.0000000000000002
7Sims: 0.12380952380952381
7Sims: 0.1111111111111111
8Sims: 0.31596034702801157
8Sims: 0.16333943833943834
8Sims: 1.0
8Sims: 0.24980663186231308
8Sims: 0.16333943833943834
8Sims: 1.0
8Sims: 0.24980663186231308
8Sims: 0.16333943833943834
9Sims: 0.3885529314328459
9Sims: 0.0
9Sims: 0.0
9Sims: 0.8091120298173655
9Sims: 0.0
9Sims: 0.0
9Sims: 

"It's been a long time no see. I can't wait to see you again.It's been a long time no see. I can't wait to see you again.It's been a long time no see. It's been a long day, without you my friend."

In [ ]:
%%time
find_similarities(sample_3.split(". "))

1Sims: 0.16977133129565256
2Sims: 0.14803946053946054
2Sims: 0.20966518012904253
3Sims: 0.20174156725627315
3Sims: 0.5177878440741008
3Sims: 0.289478125087949
4Sims: 0.15714008214008215
4Sims: 0.13219652954698724
4Sims: 0.23107928299544916
4Sims: 0.24775656226087628
5Sims: 0.1737436855083914
5Sims: 0.19010971699365573
5Sims: 0.20976050398247217
5Sims: 0.30920037190888583
5Sims: 0.1711569435466494
6Sims: 0.16801831338596046
6Sims: 0.0810249991262102
6Sims: 0.11785216793440477
6Sims: 0.21174056312367887
6Sims: 0.15612660568800918
6Sims: 0.12646481678795923


In [26]:
import pandas as pd
a= pd.read_excel("people.xlsx")
summary=[]
for (name,text) in zip(a['Full Name'],a['Biography']):
    print(name)
    try:
        summary.append((name,build_summary(text)))
    except:
        summary.append((name,''))

Bill Gates
[[1.         0.12400251 0.         ... 0.08499636 0.03426501 0.12377092]
 [0.12400251 1.         0.         ... 0.08095238 0.0875     0.16720085]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.08499636 0.08095238 0.         ... 1.         0.10285714 0.19641528]
 [0.03426501 0.0875     0.         ... 0.10285714 1.         0.24969124]
 [0.12377092 0.16720085 0.         ... 0.19641528 0.24969124 1.        ]]
[9, 17, 18, 20, 25, 29]
30
Adam Neumann
[[1.         0.49733958 0.39805885 0.21008577 0.19010152 0.17973986]
 [0.49733958 1.         0.23726326 0.17659775 0.40351602 0.12312675]
 [0.39805885 0.23726326 1.         0.17076846 0.13594395 0.14509921]
 [0.21008577 0.17659775 0.17076846 1.         0.14980879 0.22323603]
 [0.19010152 0.40351602 0.13594395 0.14980879 1.         0.24331232]
 [0.17973986 0.12312675 0.14509921 0.22323603 0.24331232 1.        ]]
[0, 1, 4]
6
Stephen Curry
[[1.         0.34749757 0.36969983 0.10611373 0.28392935 0.38216

[[1.         0.33692421 0.14045553 0.35375892]
 [0.33692421 1.         0.08971808 0.18513993]
 [0.14045553 0.08971808 1.         0.27420786]
 [0.35375892 0.18513993 0.27420786 1.        ]]
[0, 3]
4
Elizabeth Warren
[[1.         0.41624356 0.172878   0.16292564 0.14864373 0.23031052
  0.4959941  0.17269698 0.45041884 0.31287873 0.11846906 0.16637119
  0.30466015 0.03329346 0.31330828 0.18351521 0.16776291 0.08596491
  0.1433558  0.17959964 0.2510234  0.03863636]
 [0.41624356 1.         0.33382622 0.14967285 0.1202652  0.25408826
  0.20407689 0.14846978 0.35818038 0.17855104 0.13234176 0.15539513
  0.23949461 0.01592772 0.3117395  0.1087561  0.09534466 0.0588694
  0.09155381 0.10369142 0.1146278  0.0202381 ]
 [0.172878   0.33382622 1.         0.24121019 0.41155313 0.28945435
  0.09215052 0.18304563 0.21951908 0.14895422 0.07672402 0.14927221
  0.14907708 0.02345438 0.14485634 0.08560997 0.06716608 0.05875576
  0.09177647 0.07676946 0.10635024 0.02913534]
 [0.16292564 0.14967285 0.2412101

[[1.         0.03125    0.20958677 0.20754098 0.24322374 0.2044604
  0.15889537 0.29235343]
 [0.03125    1.         0.09808102 0.11711798 0.17869434 0.11340417
  0.09270306 0.10381983]
 [0.20958677 0.09808102 1.         0.14087793 0.39025963 0.24848451
  0.26480486 0.27148505]
 [0.20754098 0.11711798 0.14087793 1.         0.18473983 0.19276417
  0.07343712 0.16809882]
 [0.24322374 0.17869434 0.39025963 0.18473983 1.         0.23959722
  0.20877254 0.33252611]
 [0.2044604  0.11340417 0.24848451 0.19276417 0.23959722 1.
  0.06903236 0.23855107]
 [0.15889537 0.09270306 0.26480486 0.07343712 0.20877254 0.06903236
  1.         0.40425194]
 [0.29235343 0.10381983 0.27148505 0.16809882 0.33252611 0.23855107
  0.40425194 1.        ]]
[2, 4, 7]
8
Kunal Bahl
[[1.         0.1068536  0.18626174 0.24764253]
 [0.1068536  1.         0.16088228 0.14447834]
 [0.18626174 0.16088228 1.         0.15337623]
 [0.24764253 0.14447834 0.15337623 1.        ]]
[0, 3]
4
Daniel Curran
[[1.         0.14517196 0.161

[[1.         0.1424795  0.12082286 0.22450862 0.11454294 0.1856075
  0.18347635 0.22408555 0.12524626 0.09838294 0.12333368 0.2355001
  0.17218047]
 [0.1424795  1.         0.12162968 0.30245243 0.15994189 0.10320513
  0.18407068 0.18695182 0.17042044 0.07282301 0.15043583 0.20301591
  0.09619309]
 [0.12082286 0.12162968 1.         0.15110505 0.12791098 0.05627104
  0.20961903 0.15205467 0.1138422  0.14476056 0.12729969 0.12434714
  0.07441618]
 [0.22450862 0.30245243 0.15110505 1.         0.12282832 0.08388889
  0.69306806 0.19902236 0.16567459 0.05952381 0.13189033 0.14679427
  0.09864268]
 [0.11454294 0.15994189 0.12791098 0.12282832 1.         0.05870536
  0.12054373 0.14867465 0.11257538 0.05078125 0.28022879 0.13025077
  0.06963755]
 [0.1856075  0.10320513 0.05627104 0.08388889 0.05870536 1.
  0.13855311 0.16939776 0.12787213 0.11279762 0.15968254 0.18949653
  0.16111111]
 [0.18347635 0.18407068 0.20961903 0.69306806 0.12054373 0.13855311
  1.         0.28314608 0.26936395 0.13740

[[1.         0.06880952 0.13823675 0.26244322 0.21406803 0.25913477
  0.         0.18807304 0.13709876 0.38641327 0.14870972 0.37598017
  0.33960314 0.36091834 0.20984104]
 [0.06880952 1.         0.17218196 0.17086081 0.20821306 0.17181818
  0.         0.16648378 0.16658741 0.31335699 0.14738927 0.15344203
  0.1856111  0.18912692 0.22555745]
 [0.13823675 0.17218196 1.         0.18901635 0.27880267 0.18900992
  0.         0.21150794 0.16194139 0.17131648 0.16129196 0.99331067
  0.24124325 0.31083838 0.26320589]
 [0.26244322 0.17086081 0.18901635 1.         0.18045613 0.39305349
  0.         0.14885699 0.09774213 0.11965746 0.11258847 0.15134668
  0.19003774 0.27598573 0.21629114]
 [0.21406803 0.20821306 0.27880267 0.18045613 1.         0.10121784
  0.         0.17177293 0.16357937 0.2315447  0.08826525 0.08672161
  0.16430933 0.31897336 0.1786367 ]
 [0.25913477 0.17181818 0.18900992 0.39305349 0.10121784 1.
  0.         0.26045444 0.10537426 0.13798885 0.12664733 0.16909016
  0.20312326

...

[[1.         0.025      0.2226133  0.08920304 0.12772727 0.27733959
  0.19516612 0.27888153 0.24226481 0.15385494 0.29293939 0.12934774
  0.21009991 0.16065862 0.27967114 0.19125156 0.06788677 0.09453625]
 [0.025      1.         0.12228542 0.09608599 0.12450339 0.13331765
  0.13495126 0.17091209 0.13688878 0.13820783 0.13652328 0.13173325
  0.12635966 0.10627923 0.13152585 0.13834034 0.1515812  0.1133463 ]
 [0.2226133  0.12228542 1.         0.18860702 0.17498561 0.19780535
  0.16599102 0.21477191 0.25712448 0.14779785 0.39152412 0.11455338
  0.22668258 0.27831531 0.16339694 0.06060049 0.06794872 0.20111935]
 [0.08920304 0.09608599 0.18860702 1.         0.12089483 0.27295514
  0.1987859  0.28678817 0.23763442 0.21327279 0.44927362 0.16756749
  0.29474628 0.44637208 0.18367107 0.09444444 0.12282821 0.15654762]
 [0.12772727 0.12450339 0.17498561 0.12089483 1.         0.49387557
  0.27138149 0.36451233 0.38322822 0.1647399  0.24984151 0.13987098
  0.17360239 0.23966132 0.21536116 0.0962963

[[1.         0.10022624 0.18731962 0.06666667 0.17351201 0.20714815
  0.10453042 0.20071627 0.06666667 0.13156376 0.14373257]
 [0.10022624 1.         0.15387837 0.21589904 0.139406   0.12310809
  0.34061838 0.10863411 0.35675164 0.09242424 0.15616492]
 [0.18731962 0.15387837 1.         0.02083333 0.60362088 0.16396102
  0.15664167 0.13729224 0.02083333 0.10713482 0.15524437]
 [0.06666667 0.21589904 0.02083333 1.         0.12996074 0.12275583
  0.31500411 0.10745723 1.03680612 0.15064103 0.14961658]
 [0.17351201 0.139406   0.60362088 0.12996074 1.         0.37244143
  0.56132881 0.1553695  0.02380952 0.14015093 0.18241607]
 [0.20714815 0.12310809 0.16396102 0.12275583 0.37244143 1.
  0.12390649 0.12021021 0.02380952 0.04       0.18345925]
 [0.10453042 0.34061838 0.15664167 0.31500411 0.56132881 0.12390649
  1.         0.15257074 0.41574075 0.07638889 0.18268052]
 [0.20071627 0.10863411 0.13729224 0.10745723 0.1553695  0.12021021
  0.15257074 1.         0.07142857 0.2376412  0.30010007]


[[1.         0.20934588 0.58694767 0.44246084 0.01602564]
 [0.20934588 1.         0.14314803 0.19966814 0.02357143]
 [0.58694767 0.14314803 1.         0.44923531 0.01259851]
 [0.44246084 0.19966814 0.44923531 1.         0.00721154]
 [0.01602564 0.02357143 0.01259851 0.00721154 1.        ]]
[0, 2, 3]
3
Anne Pramaggiore
[[1.         0.16993855 0.34936885 0.32519187 0.08847328 0.2715647
  0.19276323 0.18018656]
 [0.16993855 1.         0.4661393  0.28152357 0.16847868 0.24689846
  0.34079613 0.18608116]
 [0.34936885 0.4661393  1.         0.52356028 0.3388151  0.19788648
  0.1969097  0.29484419]
 [0.32519187 0.28152357 0.52356028 1.         0.10680008 0.18128386
  0.11603097 0.12661054]
 [0.08847328 0.16847868 0.3388151  0.10680008 1.         0.23204182
  0.24850512 0.35213719]
 [0.2715647  0.24689846 0.19788648 0.18128386 0.23204182 1.
  0.52908858 0.29203016]
 [0.19276323 0.34079613 0.1969097  0.11603097 0.24850512 0.52908858
  1.         0.32170406]
 [0.18018656 0.18608116 0.29484419 0.1

[[1.         0.14219977 0.14148673 0.13650601 0.14698338 0.08410708
  0.23544671]
 [0.14219977 1.         0.15156861 0.2154889  0.18317962 0.16214082
  0.1482136 ]
 [0.14148673 0.15156861 1.         0.20465273 0.1284023  0.29358648
  0.16169167]
 [0.13650601 0.2154889  0.20465273 1.         0.39459302 0.18897403
  0.15498473]
 [0.14698338 0.18317962 0.1284023  0.39459302 1.         0.18676366
  0.37850057]
 [0.08410708 0.16214082 0.29358648 0.18897403 0.18676366 1.
  0.16352637]
 [0.23544671 0.1482136  0.16169167 0.15498473 0.37850057 0.16352637
  1.        ]]
[3, 4, 6]
7
Kevin McCarthy
[[1.         0.57193023 0.32252245 0.14690935 0.15401194 0.12065547
  0.13505499 0.11592971]
 [0.57193023 1.         0.3500974  0.21499667 0.1750743  0.09494977
  0.04668047 0.0716654 ]
 [0.32252245 0.3500974  1.         0.35852614 0.2121633  0.11314133
  0.12286601 0.11583763]
 [0.14690935 0.21499667 0.35852614 1.         0.34335969 0.24445813
  0.08775101 0.10220409]
 [0.15401194 0.1750743  0.2121633 

[[1.         0.35820732 0.23298148 0.23444492 0.26328949 0.09920829
  0.08927683 0.20185613 0.11293732 0.1025871 ]
 [0.35820732 1.         0.40763314 0.45383672 0.40799858 0.10129355
  0.17964526 0.28020648 0.11113551 0.11894653]
 [0.23298148 0.40763314 1.         0.22032661 0.21375361 0.07090803
  0.10082302 0.15537167 0.08179606 0.0874054 ]
 [0.23444492 0.45383672 0.22032661 1.         0.35414753 0.1284772
  0.14464326 0.23803441 0.12657051 0.12487196]
 [0.26328949 0.40799858 0.21375361 0.35414753 1.         0.17168432
  0.15099871 0.22206947 0.17736682 0.15177609]
 [0.09920829 0.10129355 0.07090803 0.1284772  0.17168432 1.
  0.26137644 0.14461737 0.16582785 0.12989506]
 [0.08927683 0.17964526 0.10082302 0.14464326 0.15099871 0.26137644
  1.         0.21197846 0.14202166 0.16340994]
 [0.20185613 0.28020648 0.15537167 0.23803441 0.22206947 0.14461737
  0.21197846 1.         0.1721709  0.18641074]
 [0.11293732 0.11113551 0.08179606 0.12657051 0.17736682 0.16582785
  0.14202166 0.172170

In [27]:
def toExcel(df, name):
    writer = pd.ExcelWriter(name+'.xlsx')
    df.to_excel(writer,'Sheet1')
    
toExcel(pd.DataFrame(summary),'SummariesAllNNJJ')